In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import pickle
from scipy.stats import skew
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import model_selection

In [2]:
from functions import UserInputs2WorkingForm
from functions import GetTrainingData
from functions import GetSplitTrainingData
from functions import GetAllModelsForComparison
from functions import GetTESTDataFrameNames
from functions import getTESTDataFrame
from functions import getBarPlot
from functions import getGraphs
from functions import getPlot
from functions import GenerateComparisonResultFiles
from functions import FeatureComparison
from functions import GetOnlyTwoModelsForComparison
from functions import GetFinalEightModelsForComparison
from functions import GetFinalModelForComparison
from functions import TrainModel
from functions import PredictModel
from functions import PredictProbModel
from functions import GenerateIMSDictionary
from functions import GenerateTrainingFile
from functions import Magnitude


In [3]:
"""
Generate a Training Data File
"""
GenerateTrainingFile("NotNormalized.csv")
print('finished')

finished


In [4]:
"""
Veryifies the new generate training file works as before
"""
UserInput = GenerateIMSDictionary('2003.10.22.12.06.24',"DELETE.csv",os.getcwd())
Test = GenerateIMSDictionary('2003.10.22.12.06.24',"TEST.csv",os.getcwd())

XX, YY, ALL = GetTrainingData(UserInput)
XXX, YYY, ALL1 = GetTrainingData(Test)

print(np.array_equal(XX,XXX))
print(np.array_equal(YY,YYY))
print(np.array_equal(ALL,ALL1))
print('finished')

UnboundLocalError: local variable 'dataset' referenced before assignment

In [ ]:
"""
'Simulate' receiving GUI Input
"""
UserInput = GenerateIMSDictionary('2003.10.22.12.06.24',"NoNegatives.csv",os.getcwd())
print('finished')

In [ ]:
"""
Get Training data sets
"""
X_train, X_test, Y_train, Y_test = GetSplitTrainingData(UserInput)
Xall_train, Yall_train, dataset = GetTrainingData(UserInput)
print('finished')

In [ ]:
"""
Model Comparisons
"""

#Quick Demonstration
models = GetOnlyTwoModelsForComparison(X_train,Y_train)

#Initial
#models = GetAllModelsForComparison(X_train,Y_train)

#Min Samples Comparsion
"""
for x in range(1,50,2):
    print(x)
    models = GetFinalEightModelsForComparison(X_train,Y_train,MinSampleSplit=x)
    results,string,string1,time,fig = FeatureComparison(models,X_train, X_test, Y_train, Y_test,UserInput)
    Graphname = 'GraphMinSamples' + str(x) + '.pdf'
    Timename = 'TimeMinSamples' + str(x) + '.txt'
    Scoringname = 'ScoreMinSamples' + str(x) + '.txt'
    GenerateComparisonResultFiles(results,string,string1,time,fig,str1 = Graphname,\
                                  str2 = Timename,str3 = "NoGraphs.txt",str4 = Scoringname)
"""

#Number of Estimators Comparison
"""
for x in range(10,1010,10):
    print(x)
    models = GetFinalModelForComparison(X_train,Y_train, MinSampleSplit=10, Nestimators = x)
    results,string,string1,time,fig = FeatureComparison(models,X_train, X_test, Y_train, Y_test,UserInput)
    Graphname = 'GraphMinSamples' + str(x) + '.pdf'
    Timename = 'TimeMinSamples' + str(x) + '.txt'
    Scoringname = 'ScoreMinSamples' + str(x) + '.txt'
    GenerateComparisonResultFiles(results,string,string1,time,fig,str1 = Graphname,\
                                  str2 = Timename,str3 = "NoGraphs.txt",str4 = Scoringname)
"""

print('finished')

In [ ]:
"""
Demonstrate Selected Model's results using
classification report
"""
#Test on the split model
classifier0 = TrainModel(X_train, Y_train)
Y_pred,Y_pred_string = PredictModel(classifier0,X_test)
print(classification_report(Y_test,Y_pred))
print('\nfinished')

In [ ]:
"""
Train on Final Model using all of the training data

Final Model:
RandomForestClassifier(min_samples_split = 10 ,n_estimators = 200)
{   'bootstrap': True, 
    'class_weight': None, 
    'criterion': 'gini', 
    'max_depth': None, 
    'max_features': 'auto', 
    'max_leaf_nodes': None, 
    'min_impurity_decrease': 0.0, 
    'min_impurity_split': None, 
    'min_samples_leaf': 1, 
    'min_samples_split': 10, 
    'min_weight_fraction_leaf': 0.0, 
    'n_estimators': 200, 
    'n_jobs': 1, 
    'oob_score': False, 
    'random_state': None, 
    'verbose': 0, 
    'warm_start': False}
"""
classifier = TrainModel(Xall_train, Yall_train)

print('finished')

In [ ]:
# Save Model Using Pickle
filename = 'finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

print('finished')

In [ ]:
"""
Generate a File That needs to be question
"""
X_in_question = getTESTDataFrame(UserInput)

print('finished')

In [ ]:
# load the model from disk
classifier2 = pickle.load(open(filename, 'rb'))

print('finished')

In [ ]:
"""
Prediction Results
Compared Both Current and Saved Models


"""
X_verify = X_in_question.values[:,0:(X_in_question.shape[1]-1)]

prediction,prediction_string = PredictModel(classifier,X_verify)
prediction_proba = PredictProbModel(classifier,X_verify)
print('Current Model:\n')
print(prediction)
print(prediction_string)
print(prediction_proba)

prediction,prediction_string = PredictModel(classifier2,X_verify)
prediction_proba = PredictProbModel(classifier2,X_verify)
print('\n')
print('Loaded Model:\n')
print(prediction)
print(prediction_string)
print(prediction_proba)

print('finished')

In [ ]:
"""
BEGIN Feature Reduction

"""
m = classifier.feature_importances_
m1 = GetTESTDataFrameNames(UserInput)
Z = [x for _,x in sorted(zip(m,m1))]
Z1 = sorted(m)
fig = getBarPlot(Z1[-10:],Z[-10:],"Relative Importance","FINAL")


In [ ]:
"""
Reduce Features on the Training Data and Test Data frame
"""
filename = 'NoNegatives.csv'
dataset = pd.read_csv(filename,header = 0,index_col = 0)
StateTrain = dataset['State'].values
ReducedFeatureTrainingData = {
    '2':  dataset[['RMS','FTF']],
    '3':  dataset[['RMS','FTF','Max ABS']],
    '4':  dataset[['RMS','FTF','Max ABS','Skew']],
    '5':  dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4']],
    '6':  dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4','Autocorrelate Time @ Peak 5']],
    '7':  dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4','Autocorrelate Time @ Peak 5','PSD Frq @ Peak 5']],
    '8':  dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4','Autocorrelate Time @ Peak 5','PSD Frq @ Peak 5','FFT Frq @ Peak 1']],
    '9':  dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4','Autocorrelate Time @ Peak 5','PSD Frq @ Peak 5','FFT Frq @ Peak 1','Min']],
    '10': dataset[['RMS','FTF','Max ABS','Skew','FFT Amp @ Peak 4','Autocorrelate Time @ Peak 5','PSD Frq @ Peak 5','FFT Frq @ Peak 1','Min','PSD Frq @ Peak 1']],
    '3a': dataset[['RMS','Max ABS','Skew']],
    '4a': dataset[['RMS','Max ABS','Skew','Max']],
    '5a': dataset[['RMS','Max ABS','Skew','Max','Min']],
    'all': dataset.drop(columns=['State'])
}

validation_size = 0.20
seed = 6

for set in ReducedFeatureTrainingData:
    FeatureTrain = ReducedFeatureTrainingData[set].values
    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(FeatureTrain, StateTrain, test_size=validation_size, random_state=seed) 

    Classifier = RandomForestClassifier(min_samples_split = 10 ,n_estimators = 200).fit(xtrain,ytrain)
    ypred,ypredstring = PredictModel(Classifier,xtest)
    print('Feature Number ({}):'.format(set))
    print(classification_report(ytest,ypred))
    print('\n')
    
print('finished')


In [ ]:
"""
BEGIN CODE FOR ACTUAL DATA

This will be validated using strictly the RMS, Skew values

This section generates and saves the Machine Learning model

This section is not to be run on the actual microcontroller. 
Will through an error is the training data NoNegatives.csv is not present

"""

filename = 'NoNegatives.csv'
dataset = pd.read_csv(filename,header = 0,index_col = 0)
dataset1 = dataset[['RMS','Skew']]
FeatureTrain = dataset1.values
StateTrain = dataset['State'].values


Classifier = RandomForestClassifier(min_samples_split = 10 ,n_estimators = 200).fit(FeatureTrain,StateTrain)

filename = 'ShortModel.sav'
pickle.dump(Classifier, open(filename, 'wb'))

print('finished')

In [ ]:
"""
Working on actaul data
"""

#Import Raw Data
filename = 'AccelerometerActualData.csv'
dataset = pd.read_csv(filename, header = None, index_col = False)
dataset.rename(columns={0: "Time", 1: "Value"}, inplace = True)
print(dataset.head(20))

#Prepare Raw Data
i = 0
time = []
amp = []
while i < len(dataset['Time'].values):
    i += 1
    
    if dataset['Time'].values[i] == 0 and dataset['Value'].values[i] == 0:
        break
    else:
        time.append(dataset['Time'].values[i])
        amp.append(dataset['Value'].values[i])
            
#Normalize
amp = amp - np.mean(amp)
mag_amp = Magnitude(amp)
amp = amp / mag_amp

#Set Up Test Array
Test_RMS = np.mean(amp**2)
Test_Skew = skew(amp)
Test_Combined = np.column_stack((Test_RMS,Test_Skew))

print('Finished')

In [ ]:
#Load the saved model
ClassifierShort = pickle.load(open('ShortModel.sav', 'rb'))

#Predict model
prediction,prediction_string = PredictModel(ClassifierShort,Test_Combined)
prediction_proba = PredictProbModel(ClassifierShort,Test_Combined)
print(prediction)
print(prediction_string)
print(prediction_proba)

print('finished')

In [ ]:
"""
Demonstrate Graphing Capabilites 
Combines the GraphDemonstration graphs for a single sample
"""
figs = getGraphs(UserInput)

print('finished')

In [ ]:
"Graph for Brendan"
figs = []
filename = 'NoNegatives.csv'
dataset = pd.read_csv(filename,header = 0,index_col = 0)
figs.append(getPlot(dataset.index[0:2155],dataset["RMS"][0:2155].values,"Sample # (Bearing 1, X)","RMS","RMS"))
figs.append(getPlot(dataset.index[2156:4311],dataset["RMS"][2156:4311].values,"Sample # (Bearing 2, X)","RMS","RMS"))
figs.append(getPlot(dataset.index[4312:6467],dataset["RMS"][4312:6467].values,"Sample # (Bearing 3, X)","RMS","RMS"))
figs.append(getPlot(dataset.index[6468:8623],dataset["RMS"][6468:8623].values,"Sample # (Bearing 4, X)","RMS","RMS"))

print('finished')